In [1]:
#https://python.plainenglish.io/python-pull-gcp-cloud-storage-file-into-aws-s3-6f6cb8c144b2
# Install the google-cloud-bigquery and pydata-google-auth packages
# to use this code snippet.
import google.cloud.bigquery
import google.cloud.bigquery.magics
#import pydata_google_auth
#for google storage
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.oauth2 import service_account

In [2]:
import pandas as pd
import io, numpy as np, re, os, sys, json

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="generated-atlas-345415-ee603c499617.json"

In [4]:
#local variables for credentials
service_account_json = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
#Set your GCP project Name
BgProject="Snow Cast"
#Construct a BigQuery client object.
bq_client = bigquery.Client()
gcpbucket = storage.Client().get_bucket("snowcast_maindata")
#set the your S3 bucket name
s3bucket="my-aws-s3-bucket-name"
#Configure the magics to use your credentials and project.
google.cloud.bigquery.magics.context.project = BgProject
google.cloud.bigquery.magics.context.credentials = service_account_json
credentials = service_account.Credentials.from_service_account_file(service_account_json)
client = bigquery.Client(credentials= credentials,project=BgProject)

In [5]:
#this function will be returned dataframe 
#from gcp google storage file
# gcpbucket: gcp bucket name, filename: gcp file name
def load_gsdatafile(gcpbucket, filename):
    error='-1'
    try: 
        bucket=gcpbucket.name
        filename = filename
        gsuri = 'gs://{}/{}'.format(bucket, filename) 
        df = pd.read_csv(gsuri,  encoding='utf-8')
        df.head()
        return df
    except Exception as e:
        return error

In [6]:
#this function will load dataframe into S3 bucket
#df: dataframe, s3bucket: S3 bucket name, file_key: file name
def savefile_toS3(df, s3bucket, file_key):
    msg='-1'
    try: 
        bucket=s3bucket
        file_key = file_key
        s3uri = 's3://{}/{}'.format(bucket, file_key) 
        print(s3uri) 
        df.to_csv(s3uri)
        msg='Files has been saved successfully!'
        return msg
    except Exception as e:
        return msg

In [7]:
filepath = "allregions_first.csv"

In [8]:
gdf=load_gsdatafile(gcpbucket,filepath)

In [9]:
gdf

,cell_id,date,SWE,region,geometry,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,...,datestring,centroid,center_lat,center_long,MOD10A1_SnowCover,MOD10A1_Albedo,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI
0,ASO_50M_SWE_USCARC_8,2017-07-17,48.561301,sierras,POLYGON ((-119.27063272296992 37.7439075502243...,4.697841,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13276514.66403197 3884236.855922926),37.739402,-119.264960,0.761684,0.367174,0.766883,0.814966,0.345533,0.818492
1,ASO_50M_SWE_USCARC_9,2017-07-17,33.924218,sierras,POLYGON ((-119.27035750488221 37.7349013266373...,4.736198,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13276484.103364434 3883445.3167475215),37.730396,-119.264686,0.666698,0.326389,0.669003,0.790095,0.339398,0.792235
2,ASO_50M_SWE_USCARC_26,2017-07-17,31.831418,sierras,POLYGON ((-119.25983912243692 37.7621381423023...,5.221427,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13275312.971020248 3885838.8167833057),37.757633,-119.254165,0.717794,0.387605,0.721743,0.660893,0.377931,0.665706
3,ASO_50M_SWE_USCARC_27,2017-07-17,41.723133,sierras,POLYGON ((-119.25956503213258 37.7531318793437...,5.322858,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13275282.53596653 3885047.4658173174),37.748627,-119.253892,0.758141,0.388365,0.761231,0.754647,0.370129,0.758559
4,ASO_50M_SWE_USCARC_28,2017-07-17,42.693821,sierras,POLYGON ((-119.2592910638845 37.74412560089217...,5.377803,4.685963,1.381988,18.482857,0.573913,...,2017198,POINT (-13275252.114465987 3884256.018790994),37.739620,-119.253619,0.732553,0.393211,0.737283,0.753537,0.346098,0.757798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,ASO_50M_SWE_USCASJ_4981,2017-07-19,0.000000,sierras,POLYGON ((-119.17846902476056 37.4841424587982...,1.007890,0.895925,0.796273,1.875901,0.000000,...,2017200,POINT (-13266257.240024392 3861368.6788548427),37.479637,-119.172816,0.000000,0.000000,-0.379545,0.000000,0.000000,-0.008149
29996,ASO_50M_SWE_USCASJ_4982,2017-07-19,0.000000,sierras,POLYGON ((-119.17820743284678 37.4751352738004...,0.989419,0.895925,0.796273,1.875901,0.000000,...,2017200,POINT (-13266228.195244668 3860574.332650505),37.470630,-119.172556,0.000000,0.000000,-0.399880,0.000000,0.000000,-0.046540
29997,ASO_50M_SWE_USCASJ_4983,2017-07-19,0.000000,sierras,POLYGON ((-119.17794595733588 37.4661280734780...,0.970604,0.895925,0.796273,1.875901,0.000000,...,2017200,POINT (-13266199.163389271 3859779.8909889017),37.461623,-119.172295,0.000000,0.000000,-0.413352,0.000000,0.000000,-0.123204
29998,ASO_50M_SWE_USCASJ_4984,2017-07-19,0.000000,sierras,POLYGON ((-119.17768459817323 37.4571208578330...,0.951543,0.895925,0.796273,1.875901,0.000000,...,2017200,POINT (-13266170.144452142 3858985.3538896106),37.452616,-119.172034,0.000000,0.000000,-0.396902,0.000000,0.000000,-0.128529
